In [2]:
import os
import re
import json
import pickle
import sqlite3
import jsonlines
from collections import Counter
from unicodedata import normalize
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
# import tensorflow as tf
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.regularizers import l1, l2
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import Sequence
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
# from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,ConfusionMatrixDisplay





# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# split big jsonl files to chunks

In [9]:
datasets = os.listdir(r"C:\Users\Saba\Desktop\Python\AI CS hackathon btu\datasets\websites\comprehensive_dataset")
def replace_unnecessary_symbols(text):
    filters = '1234567890!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r\b'
    translate_dict = {c: " " for c in filters}
    text = text.lower()
    # Create a translation table
    translation_table = str.maketrans(translate_dict)
    html_translate_dict = {"href":" ",
                           "src": " ",
                           " vc ":" ",
                           "jpg":" ",
                          " en ":" ",
                          "thumb":" ",
                          'title':" ",
                          " com ": " ",
                          " https ":" ",
                          " ru ":" ",
                          " net ": " ",
                          "\u00a9":" ",
                          " tags ":" ",
                          " hd ":" ",
                          '\u0308':" "}
    # html_translation_table = str.maketrans(html_translate_dict)
    # Use translate method
    new_text = text.translate(translation_table)
    for key in html_translate_dict:
        new_text = new_text.replace(key,html_translate_dict[key])
    new_text = re.sub(r'\s+', ' ', new_text)
    return new_text
def save_json(filepath_to_save,json_index,dict,name):
    with open(f'{filepath_to_save}/{json_index}_{name}_cleaned.json', 'w') as f:
        json.dump(dict, f, indent=4)
def save_to_chunks(name,file):
    
    chunks = pd.read_json(file, lines=True, chunksize=1)
    num = 1
    for chunk in chunks:
        try:
            html_content = chunk["html"].values[0]
            soup  = BeautifulSoup(html_content, 'html')
        except:
            print(f"could not parse row number {num}")
            continue
        
        for tag in soup(['script', 'style', 'img',"input",'iframe']):
            tag.decompose()
        for a_tag in soup.find_all('a'):
            del a_tag['href']
        for tag in soup.find_all(True):  # True selects all tags
            if 'style' in tag.attrs:
                del tag.attrs['style'] 
            if 'class' in tag.attrs:
                del tag.attrs['class']
            if 'target' in tag.attrs:
                del tag.attrs['target']
                
        # we should get rid of tab and escape charaters, also weird symbols and replace duplicate spaces with one space
        html_text = soup.get_text(separator=' ')
        html_text = normalize('NFKD', html_text)
        
        html_text = replace_unnecessary_symbols(html_text)
        
        url = normalize('NFKD', chunk['url'].values[0])
        label = chunk['label'].values[0]
        
        processed_json = {
            "url":url,
            "label":label,
            "html":html_text
        }
        filepath_to_save = 'C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_chunks_cleaned'
        save_json(filepath_to_save,num,processed_json,name)
        if num % 50 == 0:
            print(f"{num}/{20000}", end='\r')
            if num % 1000 == 0:
                break
        num +=1
        
    

In [16]:
  
file = "C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_dataset/"+datasets[1]
print(file)

C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_dataset/benign_OWS_URL_HTML_DS_part_1.jsonl


In [17]:
save_to_chunks("benign",file)

# Load chunks of datasets

In [25]:
def load_chunk_to_dataframe(chunk_index):
    chunks_filepath = "C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_chunks_parsed/"
    chunks = os.listdir(chunks_filepath)
    chunk_fullpath = chunks_filepath+chunks[chunk_index]
    df = pd.read_json(chunk_fullpath)
    # df.drop(columns=["sublabel",'crawled','uid'],inplace=True)
    return df



In [26]:
chunks_filepath = "C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_chunks_parsed/"
chunks = os.listdir(chunks_filepath)
chunks_length = len(chunks)
# print(chunks_length)
json_files_full = [chunks_filepath+chunk for chunk in chunks]